# Olivine-Spinel-Melts-$f_{O_2}$ Calculation

Welcome to the code for the olivine-spinel-melt oxygen barometer.  

This code uses the MELTS algorithm to calculate the oxygen fugacity based on the chemical potentials of 
olivine, spinel and aSiO2melt using temperatures, pressures, olivine, spinel, and whole rock 
(or glass or melt inclusions)compositions that you upload as a single excel spreadsheet that you can find in the data repository of American Mineralogist. Please do not change the headers in the spreadsheet or the upload code will not work.

# HOW TO DOWNLOAD THE LOG FO2 VALUES (PLEASE READ)

We've designed this spreadsheet so that it outputs an abundant amount of information of phase activity and chemical potentials for users to access. If a user is only interested in the log fO2 values, then the user should to run all cells in the code within the ENKI cluster (after uploading a workbook containing the olvine-spinel-liquid sets) look for two excel or csv files named:

MELTSOSaS_OSaS_Benchmark_Input_headers

and

Classic_OSaS_OSaS_Benchmark_Input_headers

The file "MELTSOSaS_OSaS_Benchmark_Input" contains the log fO2 values associated with the MELTS only implementation of the OSaS equilibrium, as well as the chemical potentials of all associated phases used in the calculation of fO2. 

The file "Classic_OSaS_OSaS_Benchmark_Input" contains the log fO2 values associated with the Classical implementation of the OSaS equilbirium (mixture of activity models). This file also contains the Fo contents of olivine, the cations per formula unit of spinel, Fe-Mg Kd values for olivine based on both the Classic OSaS and MELTs OSaS derived values of fO2, the Fe-MgKD values for spinel-olivine, and the predicted Fe-MgKD values for spinel-olivine. 

# OTHER OUTPUTS:

Ol__calcs_OSaS_Benchmark_Input_headers: File contains calculations of moles, endmembers, chemical potentials and activities associated with the olivine compositions entered in the input file.

Spin__calcs_OSaS_Benchmark_Input_headers: File contains calculations of moles, endmembers, chemical potentials and activities associated with the spinel compositions entered in the input file.

Liq_calcs_OSaS_Benchmark_Input_headers: File contains calculations of moles of endmembers used in the melts algorithm, the chemical potential of SiO2, the affinity of SiO2 in the liquid referenced to tridymite and the affinity of SiO2 in the liquid referenced to quartz.

Finally, close inspection of the code will reveal that many lines have save text file commands that are commented out. If a user is interested in other outputs, they are welcome to uncomment lines of code and explore outputs, please note that these text files may not have headers, so attention must be paid to the command lines and comments.




In [1]:
from __future__   import division
import numpy      as np
from   scipy      import optimize as optim
import pandas     as pd
import ipywidgets as ipw

## Upload that data
- In this cell, we will use pandas (any command that is followed by a .pd) to upload all your data at once. 
- You can use the spreadsheet "OSaS_Benchmark_Input.xlsx" as a reference for position of values- that is if you enter your data the same way as the example, the following code will upload your data with ease. 

- PLEASE DO NOT MODIFY HEADERS.

- If you are a confident programmer, please feel free to modify locations/names of variables, though keep in mind if you change a variable name, you'll have to alter the rest, downstream.

- The code loads in data from this spreadsheet by columns of the same values, where individual ol-spin-melt sets are in rows.

In [2]:
data_frame = pd.read_excel('OSaS_MAY_Input.xlsx').fillna(0)

### Assign variables

In [3]:
Case        = data_frame['Case'].to_numpy()
Ref         = data_frame['Study'].to_numpy()
Abbrev      = data_frame['abbrev']
Expnum      = data_frame['Exp#'].to_numpy() 
logfO2      = 0.0 #If you know this from an experiment input it here; If you don't know this, if not put 0
presMpa     = data_frame['Pressure'].to_numpy()
tempc       = data_frame['Temperature'].to_numpy()

OLIVINE COMPOSITION

In [4]:
Ol_SiO2     = data_frame['SiO2 (Olivine)'].to_numpy()
Ol_TiO2     = data_frame['TiO2 (Olivine)'].to_numpy()
Ol_Al2O3    = data_frame['Al2O3 (Olivine)'].to_numpy()
Ol_Cr2O3    = data_frame['Cr2O3 (Olivine)'].to_numpy()
Ol_FeO      = data_frame['FeOT (Olivine)'].to_numpy()
Ol_MnO      = data_frame['MnO (Olivine)'].to_numpy()
Ol_MgO      = data_frame['MgO (Olivine)'].to_numpy()
Ol_CaO      = data_frame['CaO (Olivine)'].to_numpy()
Ol_NiO      = data_frame['NiO (Olivine)'].to_numpy()

SPINEL COMPOSITION

In [5]:
Spin_SiO2   = data_frame['SiO2 (Spinel)'].to_numpy()
Spin_TiO2   = data_frame['TiO2 (Spinel)'].to_numpy()
Spin_Al2O3  = data_frame['Al2O3 (Spinel)'].to_numpy()
Spin_Cr2O3  = data_frame['Cr2O3 (Spinel)'].to_numpy()
Spin_V2O3   = data_frame['V2O3 (Spinel)'].to_numpy()
# Use this as FeOT for initial import
Spin_FeOT   = data_frame['FeO (Spinel)'].to_numpy() 
Spin_MnO    = data_frame['MnO (Spinel)'].to_numpy()
Spin_MgO    = data_frame['MgO (Spinel)'].to_numpy()
Spin_CaO    = data_frame['CaO (Spinel)'].to_numpy()
Spin_ZnO    = data_frame['ZnO (Spinel)'].to_numpy()
#(print(Spin_Al2O3))

MELT COMPOSITION

In [6]:
Mlt_SiO2    = data_frame['SiO2 (Melt)'].to_numpy()
Mlt_TiO2    = data_frame['TiO2 (Melt)'].to_numpy()
Mlt_Al2O3   = data_frame['Al2O3 (Melt)'].to_numpy()
Mlt_FeOT    = data_frame['FeOT (Melt)'].to_numpy()
Mlt_MnO     = data_frame['MnO (Melt)'].to_numpy()
Mlt_MgO     = data_frame['MgO (Melt)'].to_numpy()
Mlt_CaO     = data_frame['CaO (Melt)'].to_numpy()
Mlt_Na2O    = data_frame['Na2O (Melt)'].to_numpy()
Mlt_K2O     = data_frame['K2O (Melt)'].to_numpy()
Mlt_H2O     = data_frame['H2O (Melt)'].to_numpy()
Mlt_CO2     = data_frame['CO2 (Melt)'].to_numpy()

Convert Tempratures from °C to K and Pressure from MPa to bars

In [7]:
t = tempc + 273.15   #Converting temperature to kelvin 
p = presMpa*10       #Converting pressure to bars

## Import the important packages from melts

In [8]:
from thermoengine import core, phases, model

Access the database (by default, the Berman (1988)/MELTS database)

In [9]:
modelDB = model.Database()

### Retrieve the thermoengine properties of Olivine

In [10]:
Olivine = modelDB.get_phase('Ol')

Calculate olivine moles

In [11]:
#First, I set up a variable that corresponds to the number of rows in the dataset
#using a function that is based on the input dataset. This way, the variable will 
#scale correctly even if the file input changes.
olpoints = Ol_SiO2.shape[0]              

#This is the empty array where we will store the results for the 16 mol oxides for 
#olivine compositions. 
Ol_moles = np.empty([olpoints,16])

#This is a loop that calculates and stores the mol oxides for every row of data in 
#the input file.
for olpoint in range(olpoints):  
    Ol_mol_oxides = core.chem.format_mol_oxide_comp({'SiO2':Ol_SiO2[olpoint],'TiO2':Ol_TiO2[olpoint], 'Al2O3':Ol_Al2O3[olpoint], 'Cr2O3':Ol_Cr2O3[olpoint],'FeO':Ol_FeO[olpoint], 'MnO':Ol_MnO[olpoint],'MgO':Ol_MgO[olpoint],'CaO':Ol_CaO[olpoint], 'NiO':Ol_NiO[olpoint]}, convert_grams_to_moles=True)
    Ol_moles[olpoint,:] = Ol_mol_oxides[:]
    
#Remove the # to check the olivine moles in the following line of code
#print(Ol_moles)

Calculate moles of olivine endmembers

In [12]:
#Setting up a second variable that corresponds to the number of rows in the dataset using 
#a function that is based on the input dataset for correct scaling and no stored values.
ol2points = Ol_moles.shape[0]            

#This is the empty array where the code will output the six olivine endmembers
#the code will output the intrinsic endmember proportions in the following order:
#'tephroite'; 'fayalite'; 'co-olivine'; 'ni-olivine';'monticellite';'forsterite'
Ol_molend = np.empty([ol2points,6]) 

#This is a loop that will calculate and store the mole fraction of endmembers of 
#olivine in the above listed order.
for ol2point in range(ol2points):       
    moles_end,oxide_res = Olivine.calc_endmember_comp(mol_oxide_comp=Ol_moles[ol2point], method='intrinsic', output_residual=True)
    Ol_molend[ol2point,:] = moles_end[:]

#The following code creates empty arrays, calculates the sum of the oxide components 
#to calculate the olivine mole fractions of the oxide components in the following order:
#'SiO2','TiO2','Al2O3','Fe2O3','Cr2O3','FeO','MnO','MgO','NiO','CoO','CaO','Na2O','K2O','P2O5','H2O','CO2’
ol_moloxsum = np.empty([ol2points,1]) 
for ol2point in range (ol2points):
    ol_moloxsum[ol2point,0] = Ol_moles[ol2point].sum()

ol_molfrac = np.empty([ol2points,16])
XSiO2_ol   = np.empty([ol2points,1])
XTiO2_ol   = np.empty([ol2points,1])
XAl2O3_ol  = np.empty([ol2points,1])
XCr2O3_ol  = np.empty([ol2points,1]) 
XFeOT_ol   = np.empty([ol2points,1]) 
XMgO_ol    = np.empty([ol2points,1]) 
XMnO_ol    = np.empty([ol2points,1]) 
XCaO_ol    = np.empty([ol2points,1]) 

for ol2point in range (ol2points):
    ol_molfrac[ol2point,:]  = Ol_moles[ol2point,:]/ol_moloxsum[ol2point,0]
    XSiO2_ol [ol2point,0]   = ol_molfrac[ol2point,0]
    XTiO2_ol [ol2point,0]   = ol_molfrac[ol2point,1]
    XAl2O3_ol[ol2point,0]   = ol_molfrac[ol2point,2]
    XCr2O3_ol[ol2point,0]   = ol_molfrac[ol2point,4]
    XFeOT_ol [ol2point,0]   = ol_molfrac[ol2point,5]
    XMnO_ol  [ol2point,0]   = ol_molfrac[ol2point,6]
    XMgO_ol  [ol2point,0]   = ol_molfrac[ol2point,7]
    XCaO_ol  [ol2point,0]   = ol_molfrac[ol2point,10]


#The following lines of code create empty arrays, calculate, then store values of the
#moles of oxide components in olivine (note: not the mole fraction), and calculates Fo content
molSiO2_ol   = np.empty([ol2points,1])
molTiO2_ol   = np.empty([ol2points,1])
molAl2O3_ol  = np.empty([ol2points,1])
molCr2O3_ol  = np.empty([ol2points,1]) 
molFeOT_ol   = np.empty([ol2points,1]) 
molMnO_ol    = np.empty([ol2points,1]) 
molMgO_ol    = np.empty([ol2points,1]) 
molCaO_ol    = np.empty([ol2points,1]) 

molSiO2_ol   = Ol_SiO2  / 60.08
molTiO2_ol   = Ol_TiO2  / 79.866
molAl2O3_ol  = Ol_Al2O3 / 101.96
molCr2O3_ol  = Ol_Cr2O3 / 151.99
molFeOT_ol   = Ol_FeO   / 71.844
molMnO_ol    = Ol_MnO   / 70.9374
molMgO_ol    = Ol_MgO   / 40.3044
molCaO_ol    = Ol_CaO   / 56.0774 
molNiO_ol    = Ol_NiO   / 74.6928 

OlFo         = Ol_MgO/40.3044/((Ol_MgO/40.3044)+(Ol_FeO/71.844))*100
#print(molFeOT_sp)

Compute activities and chemical potentials of endmember components

In [13]:
#Setting up a second variable that corresponds to the number of rows in the dataset using 
#a function that is based on the input dataset for correct scaling and no stored values.
Ol_molend_pts  = Ol_molend.shape[0]    # number of points in mol fractions array
temp_pts       = t.shape[0]            # number of points in temperature array
pres_pts       = p.shape[0]            # number of points in pressure array

#The following code creates empty arrays in which the variables for all the olivine chemical potentials, 
#all olivine centric information (Ol_melts_calcs_all), the chemical potentials of fayalite (muFe2SiO4), 
#and the activities of fayalite (aFe2SiO4) will be stored.
Ol_chempotentials        = np.empty([temp_pts,6])                 
Ol_melts_calcs_all       = np.empty([temp_pts,(1+1+1+16+6+6+6)])
muFe2SiO4                = np.empty([temp_pts,1])
aFe2SiO4                 = np.empty([temp_pts,1]) 

#The following loop calculates and stores chemical potentials and activities of each olivine endmember
for ol3point in range(temp_pts):
    Olchempotential = Olivine.chem_potential(t[ol3point], p[ol3point], mol=Ol_molend[ol3point,:])
    Olchemactivity  = Olivine.activity      (t[ol3point], p[ol3point], mol=Ol_molend[ol3point,:])
    
    Ol_chempotentials[ol3point,:]         = Olchempotential[:]           
    Ol_melts_calcs_all[ol3point,0]        = Case[ol3point]
    Ol_melts_calcs_all[ol3point,1]        = tempc[ol3point]
    Ol_melts_calcs_all[ol3point,2]        = p[ol3point]
    Ol_melts_calcs_all[ol3point,3:19]     = Ol_moles [ol3point,:]           #16 columns for mol fractions of 16 oxides
    Ol_melts_calcs_all[ol3point,19:25]    = Ol_molend[ol3point,:]           #6 columns for end members 'tephroite' 'fayalite'
                                                                            # 'co-olivine' 'ni-olivine' 'monticellite''forsterite'
    Ol_melts_calcs_all[ol3point,25:31]    = Olchempotential[:]              #6 columns for chemical potentials for each end member 
    Ol_melts_calcs_all[ol3point,31:37]    = Olchemactivity[:]               #6 columns for chemical potentials for each end member
    aFe2SiO4[ol3point,0]                  = Ol_melts_calcs_all[ol3point,32] #Fill the empty array with the activity of fayalite
    muFe2SiO4[ol3point,0]                 = Ol_chempotentials[ol3point,1]   #Fill the array with chemical potentials for fayalite   
Ol_melts_calcs_all_df = pd.DataFrame({
    'Case':              Ol_melts_calcs_all[:, 0].astype(int),
    'T °C':              Ol_melts_calcs_all[:, 1],
    'P bars':            Ol_melts_calcs_all[:, 2],
    'Ol moles SiO2':     Ol_melts_calcs_all[:, 3],
    'Ol moles TiO2':     Ol_melts_calcs_all[:, 4],
    'Ol moles Al2O3':    Ol_melts_calcs_all[:, 5],
    'Ol moles Fe2O3':    Ol_melts_calcs_all[:, 6],
    'Ol moles Cr2O3':    Ol_melts_calcs_all[:, 7],
    'Ol moles FeO':      Ol_melts_calcs_all[:, 8],
    'Ol moles MnO':      Ol_melts_calcs_all[:, 9],
    'Ol moles MgO':      Ol_melts_calcs_all[:,10],
    'Ol moles NiO':      Ol_melts_calcs_all[:,11],
    'Ol moles CoO':      Ol_melts_calcs_all[:,12],
    'Ol moles CaO':      Ol_melts_calcs_all[:,13],
    'Ol moles Na2O':     Ol_melts_calcs_all[:,14],
    'Ol moles K2O':      Ol_melts_calcs_all[:,15],
    'Ol moles P2O5':     Ol_melts_calcs_all[:,16],
    'Ol moles H2O':      Ol_melts_calcs_all[:,17],
    'Ol moles CO2':      Ol_melts_calcs_all[:,18],
    'm tephroite':       Ol_melts_calcs_all[:,19],
    'm fayalite':        Ol_melts_calcs_all[:,20],
    'm co-olivine':      Ol_melts_calcs_all[:,21],
    'm ni-olivine':      Ol_melts_calcs_all[:,22],
    'm monticellite':    Ol_melts_calcs_all[:,23],
    'm forsterite':      Ol_melts_calcs_all[:,24],
    'mu tephroite kJ':   Ol_melts_calcs_all[:,25]/1000.0,
    'mu fayalite kJ':    Ol_melts_calcs_all[:,26]/1000.0,
    'mu co-olivine kJ':  Ol_melts_calcs_all[:,27]/1000.0,
    'mu ni-olivine kJ':  Ol_melts_calcs_all[:,28]/1000.0,
    'mu monticellite kJ':Ol_melts_calcs_all[:,29]/1000.0,
    'mu forsterite kJ':  Ol_melts_calcs_all[:,30]/1000.0,
    'a tephroite':       Ol_melts_calcs_all[:,31],
    'a fayalite':        Ol_melts_calcs_all[:,32],
    'a co-olivine':      Ol_melts_calcs_all[:,33],
    'a ni-olivine':      Ol_melts_calcs_all[:,34],
    'a monticellite':    Ol_melts_calcs_all[:,35],
    'a forsterite':      Ol_melts_calcs_all[:,36]
})
Ol_melts_calcs_all_df.to_csv("Ol__calcs_OSaS_Benchmark_Input.csv", index=False)
Ol_melts_calcs_all_df.to_excel("Ol__calcs_OSaS_Benchmark_Input.xlsx", index=False)


In [14]:
Ol_melts_calcs_all[:,0]

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.])

### Retrieve the thermoengine properties of spinel

In [15]:
Spinel = modelDB.get_phase('SplS')

In [16]:
#Calculate Spinel moles oxides
#Setting up a new variable that corresponds to the number of rows in the dataset using 
#a function that is based on the input dataset for correct scaling and no stored values.
spinpoints = Spin_SiO2.shape[0]        

#This is the empty array where we will store the results for the mol oxides for spinel compositions. 
sp_molox = np.empty([spinpoints,16])
molSiO2_sp   = np.empty([spinpoints,1])
molTiO2_sp   = np.empty([spinpoints,1])
molAl2O3_sp  = np.empty([spinpoints,1])
molCr2O3_sp  = np.empty([spinpoints,1]) 
molFeOT_sp   = np.empty([spinpoints,1]) 
molMgO_sp    = np.empty([spinpoints,1]) 
molMnO_sp    = np.empty([spinpoints,1]) 
molCaO_sp    = np.empty([spinpoints,1]) 

#This is a loop that will calculate and store the mol oxides for each row of spinel data
for spinpoint in range(spinpoints):
    sp_mol_oxides = core.chem.format_mol_oxide_comp({'SiO2':Spin_SiO2[spinpoint],'TiO2':Spin_TiO2[spinpoint], 'Al2O3':Spin_Al2O3[spinpoint], 'Cr2O3':Spin_Cr2O3[spinpoint],'FeO':Spin_FeOT[spinpoint], 'MnO':Spin_MnO[spinpoint],'MgO':Spin_MgO[spinpoint],'CaO':Spin_CaO[spinpoint]}, convert_grams_to_moles=True)
    sp_molox[spinpoint,:]   = sp_mol_oxides[:]
    molSiO2_sp[spinpoint,:] = sp_molox[spinpoint,0]
    molTiO2_sp[spinpoint,:] = sp_molox[spinpoint,1]
    molAl2O3_sp[spinpoint,:]= sp_molox[spinpoint,2]
    molCr2O3_sp[spinpoint,:]= sp_molox[spinpoint,3]
    molFeOT_sp[spinpoint,:] = sp_molox[spinpoint,5]
    molMnO_sp[spinpoint,:]  = sp_molox[spinpoint,6]
    molMgO_sp[spinpoint,:]  = sp_molox[spinpoint,7]
    molCaO_sp[spinpoint,:]  = sp_molox[spinpoint,10]

    
#The following code creates empty arrays, calculates the sum of the oxide components 
#to calculate the spinel mole fractions of the oxide components in the following order:
#'SiO2','TiO2','Al2O3','Fe2O3','Cr2O3','FeO','MnO','MgO','NiO','CoO','CaO','Na2O','K2O','P2O5','H2O','CO2’
sp_moloxsum = np.empty([spinpoints,1]) 

for spinpoint in range (spinpoints):
    sp_moloxsum[spinpoint,0] = sp_molox[spinpoint].sum()

sp_molfrac = np.empty([spinpoints,16])
XSiO2_sp   = np.empty([spinpoints,1])
XTiO2_sp   = np.empty([spinpoints,1])
XAl2O3_sp  = np.empty([spinpoints,1])
XCr2O3_sp  = np.empty([spinpoints,1]) 
XFeOT_sp   = np.empty([spinpoints,1]) 
XMgO_sp    = np.empty([spinpoints,1]) 
XMnO_sp    = np.empty([spinpoints,1]) 
XCaO_sp    = np.empty([spinpoints,1]) 

#This is a loop that will solve and save the spinel oxide mol fraction values.
for spinpoint in range (spinpoints):
    sp_molfrac[spinpoint,:] = sp_molox[spinpoint,:]/sp_moloxsum[spinpoint,0]
    XSiO2_sp[spinpoint,0]   = sp_molfrac[spinpoint,0]
    XTiO2_sp[spinpoint,0]   = sp_molfrac[spinpoint,1]
    XAl2O3_sp[spinpoint,0]  = sp_molfrac[spinpoint,2]
    XCr2O3_sp[spinpoint,0]  = sp_molfrac[spinpoint,4]
    XFeOT_sp[spinpoint,0]   = sp_molfrac[spinpoint,5]
    XMgO_sp[spinpoint,0]    = sp_molfrac[spinpoint,7]
    XMnO_sp[spinpoint,0]    = sp_molfrac[spinpoint,6]
    XCaO_sp[spinpoint,0]    = sp_molfrac[spinpoint,10]
#print(sp_moloxsum)


#The following code creates empty arrays, calculates the mols of the oxides, 
#stores them and calculates the Cr-number of the spinel
molSiO2_sp2   = np.empty([spinpoints,1])
molTiO2_sp2   = np.empty([spinpoints,1])
molAl2O3_sp2  = np.empty([spinpoints,1])
molCr2O3_sp2  = np.empty([spinpoints,1]) 
molV2O3_sp2   = np.empty([spinpoints,1])
molFeOT_sp2   = np.empty([spinpoints,1]) 
molMgO_sp2    = np.empty([spinpoints,1]) 
molMnO_sp2    = np.empty([spinpoints,1]) 
molCaO_sp2    = np.empty([spinpoints,1]) 

molSiO2_sp2   = Spin_SiO2  / 60.08
molTiO2_sp2   = Spin_TiO2  / 79.866
molAl2O3_sp2  = Spin_Al2O3 / 101.96
molCr2O3_sp2  = Spin_Cr2O3 / 151.99
molV2O3_sp2   = Spin_V2O3  / 149.88
molFeOT_sp2   = Spin_FeOT   / 71.844
molMnO_sp2    = Spin_MnO   / 70.9374
molMgO_sp2    = Spin_MgO   / 40.3044
molCaO_sp2    = Spin_CaO   / 56.0774

Crno_spin     = Spin_Cr2O3/151.99/((Spin_Cr2O3/151.99)+(Spin_Al2O3/101.96))*100

Calculate moles of Spinel endmembers

In [17]:

spin2points = sp_molox.shape[0]          #This function sets Spin2points as the value of the number of rows in sp_molox. 
                                         #I do this because it means the new arrays will always be a function of the preceeding dataset. 
#print(points)

Sp_molend = np.empty([spin2points,5])    #this is the empty array where the code will output its results, to understand the sizing here, you need to first use the shape function
                                         #to output the shape of the results. Then use the shape function to make the array the correct size based off your dataset. 
                                         #Model gives five spinel endmembers 'chromite' 'hercynite' 'magnetite' 'spinel' 'ulvospinel'
#print(test2.shape)

for spin2point in range(spin2points):    #This is a loop that will calculate and store the mols of the five spinel end members in the order listed in the preceeding comment.
    sp_moles_end,oxide_res  = Spinel.calc_endmember_comp(mol_oxide_comp=sp_molox[spin2point], method='intrinsic', output_residual=True)
    Sp_molend[spin2point,:] = sp_moles_end[:]
    
#print(spmolend)

Compute activities and chemical potentials of endmember components

In [18]:
#Setting up a new variable that corresponds to the number of rows in the dataset using 
#a function that is based on the input dataset for correct scaling and no stored values.
spin3points             = Sp_molend.shape[0]                   
temp_pts                = t.shape[0]                           
pres_pts                = p.shape[0]      

#print(spin3points)
#The following code creates empty arrays in which the variables for all the spinel chemical potentials (Sp_chempotentials), 
#all spinel centric information (Sp_melts_calcs_all), the chemical potentials of fayalite (muFe3O4), 
#and the activities of fayalite (aFe3O4) will be stored.
Sp_chempotentials       = np.empty([temp_pts,5])               
Sp_melts_calcs_all      = np.empty([temp_pts,(1+1+1+16+5+5+5)])
muFe3O4                 = np.empty([temp_pts,1])               
aFe3O4                  = np.empty([temp_pts,1]) 

#The following loop calculates and stores chemical potentials and activities of each spinel endmember
for spin3point in range(spin3points):
    Spchemactivity                          = Spinel.activity      (t[spin3point], p[spin3point], mol=Sp_molend[spin3point,:])  #Calculate the spinel activites using MELTS
    Spchempotential                 = Spinel.chem_potential(t[spin3point], p[spin3point], mol=Sp_molend[spin3point,:])  #Calculate the spinel chemical potentials using MELTS
    Sp_chempotentials[spin3point,:]         = Spchempotential[:]                  #Write the chemical potentials to the empty array we created above.
    Sp_melts_calcs_all[spin3point,0]        = Case[spin3point]                    #Start filling in the array set up for all values; Here I start with the a reference to the case number
    Sp_melts_calcs_all[spin3point,1]        = tempc[spin3point]                   #Fill in the temperature from the original input in Celcius.
    Sp_melts_calcs_all[spin3point,2]        = p[spin3point]                       #Fill in the pressure in bars.
    Sp_melts_calcs_all[spin3point,3:19]     = sp_molox[spin3point,:]              #16 columns for mol fractions of 16 oxides
    Sp_melts_calcs_all[spin3point,19:24]    = Sp_molend[spin3point,:]             #5 columns for the fraction of each of the end members'FeCr2O4' 'FeAl2O4' 'Fe3O4' 'MgAl2O4' 'Fe2TiO4'
    Sp_melts_calcs_all[spin3point,24:29]    = Spchempotential[:]                  #5 columns for chemical potentials for each end member 'FeCr2O4' 'FeAl2O4' 'Fe3O4' 'MgAl2O4' 'Fe2TiO4'
    Sp_melts_calcs_all[spin3point,29:35]    = Spchemactivity[:]                   #5 columns for chemical activities for each end member 'FeCr2O4' 'FeAl2O4' 'Fe3O4' 'MgAl2O4' 'Fe2TiO4'
    aFe3O4[spin3point,:]                    = Sp_melts_calcs_all[spin3point,31]   #Store the activity of magnetite
    muFe3O4[spin3point,0]                   = Sp_chempotentials[spin3point,2]     #Store the chemical potential of magnetite


#np.savetxt("Sp_melts_calcs_Application_NaturalSamplesv2.csv", Sp_melts_calcs_all, delimiter=",")  

#The following code applies a correction to the MELTs derived chemical potentials of magnetite
muFe3O4_correction  = -88317.116658736 + 0.968070202*muFe3O4+ 66581.745702629*XCr2O3_sp + 75747.288514812*XMgO_sp + 53625.743547049*XFeOT_sp

#print(muFe3O4_correction)
Sp_melts_calcs_all2      = np.empty([temp_pts,(1+1+1+16+5+5+5+1)])

#print(Sp_melts_calcs_all.shape)
#print(Sp_melts_calcs_all2.shape)

for spin3point in range(spin3points):
    Sp_melts_calcs_all2[spin3point,0:34]      = Sp_melts_calcs_all[spin3point,:]
    Sp_melts_calcs_all2[spin3point,34]        = muFe3O4_correction[spin3point,:]

#print(Sp_melts_calcs_all2)
#Saving to an export file (xlsx or csv with headers)
Spin_melts_calcs_all_df = pd.DataFrame({
    'Case':                      Sp_melts_calcs_all2[:, 0].astype(int),
    'T °C':                      Sp_melts_calcs_all2[:, 1],
    'P bars':                    Sp_melts_calcs_all2[:, 2],
    'Spin moles SiO2':           Sp_melts_calcs_all2[:, 3],
    'Spin moles TiO2':           Sp_melts_calcs_all2[:, 4],
    'Spin moles Al2O3':          Sp_melts_calcs_all2[:, 5],
    'Spin moles Fe2O3':          Sp_melts_calcs_all2[:, 6],
    'Spin moles Cr2O3':          Sp_melts_calcs_all2[:, 7],
    'Spin moles FeO':            Sp_melts_calcs_all2[:, 8],
    'Spin moles MnO':            Sp_melts_calcs_all2[:, 9],
    'Spin moles MgO':            Sp_melts_calcs_all2[:,10],
    'Spin moles NiO':            Sp_melts_calcs_all2[:,11],
    'Spin moles CoO':            Sp_melts_calcs_all2[:,12],
    'Spin moles CaO':            Sp_melts_calcs_all2[:,13],
    'Spin moles Na2O':           Sp_melts_calcs_all2[:,14],
    'Spin moles K2O':            Sp_melts_calcs_all2[:,15],
    'Spin moles P2O5':           Sp_melts_calcs_all2[:,16],
    'Spin moles H2O':            Sp_melts_calcs_all2[:,17],
    'Spin moles CO2':            Sp_melts_calcs_all2[:,18],
    'm chromite':                Sp_melts_calcs_all2[:,19], 
    'm hercynite':               Sp_melts_calcs_all2[:,20],
    'm magnetite':               Sp_melts_calcs_all2[:,21],
    'm spinel':                  Sp_melts_calcs_all2[:,22],
    'm ulvospinel':              Sp_melts_calcs_all2[:,23],
    'mu chromite kJ':            Sp_melts_calcs_all2[:,24]/1000.0,
    'mu hercynite kJ':           Sp_melts_calcs_all2[:,25]/1000.0,   
    'mu magnetite kJ':           Sp_melts_calcs_all2[:,26]/1000.0,
    'mu spinel kJ':              Sp_melts_calcs_all2[:,27]/1000.0,
    'mu ulvospinel kJ':          Sp_melts_calcs_all2[:,28]/1000.0,
    'a chromite':                Sp_melts_calcs_all2[:,29],
    'a hercynite':               Sp_melts_calcs_all2[:,30],
    'a magnetite':               Sp_melts_calcs_all2[:,31],
    'a spinel':                  Sp_melts_calcs_all2[:,32],
    'a ulvospinel':              Sp_melts_calcs_all2[:,33],
    'corrected mu magnetite kJ': Sp_melts_calcs_all2[:,34]/1000.0
})
Spin_melts_calcs_all_df.to_csv("Spin__calcs_OSaS_Benchmark_Input.csv", index=False)
Spin_melts_calcs_all_df.to_excel("Spin__calcs_OSaS_Benchmark_Input.xlsx", index=False)
#print(aFe2SiO4)

### Liquid

In [19]:
#Set the version of melts that you want to use, for these calculations you should use 1.02
src_obj = core.get_src_object('EquilibrateUsingMELTSv102') 

#Access the database (by default, the Berman (1988)/MELTS database)
modelDB = model.Database(liq_mod='v1.0')

#Load in the Liquid model
Liquid = modelDB.get_phase('Liq')

Calculate Liquid moles oxides

In [20]:
#Create a new variable that is a function of the input file and an empty array for storing values
liqpoints = Mlt_SiO2.shape[0]                                                
liq_molox = np.empty([liqpoints,16])

#This is a loop that will calculate and store all values for each row of liquid 
#compositions from the dataset
for liqpoint in range(liqpoints):             
    liqmoloxides = core.chem.format_mol_oxide_comp({'SiO2':Mlt_SiO2[liqpoint],'TiO2':Mlt_TiO2[liqpoint], 'Al2O3':Mlt_Al2O3[liqpoint],'FeO':Mlt_FeOT[liqpoint], 'MnO':Mlt_MnO[liqpoint],'MgO':Mlt_MgO[liqpoint],'CaO':Mlt_CaO[liqpoint],'Na2O':Mlt_Na2O[liqpoint],'K2O':Mlt_K2O[liqpoint],'H2O':Mlt_H2O[liqpoint],'CO2':Mlt_CO2[liqpoint]}, convert_grams_to_moles=True)
    liq_molox[liqpoint,:] = liqmoloxides[:]
    
#print(liq_molox) #Uncomment this value and make sure that your values look ok (e.g., XSiO2, the first entry is like >0.7 or so, composition dependent of course). Note this is mole oxide not mol fraction.

Calculate moles of Liquid endmembers

In [21]:
#The following code creates a new variable that is a function of the input file, 
#an empty array for storing values, and a loop that calculates and stores the fifteen liquid endmembers:
#'SiO2' 'TiO2' 'Al2O3' 'Fe2O3' 'MgCr2O4' 'Fe2SiO4' 'MnSi0.5O2' 'Mg2SiO4'
# 'NiSi0.5O2' 'CoSi0.5O2' 'CaSiO3' 'Na2SiO3' 'KAlSiO4' 'Ca3(PO4)2' 'H2O'

liq2points = liq_molox.shape[0]          
Liq_molend = np.empty([liq2points,15])   

for liq2point in range(liq2points):
    liq_moles_end,oxide_res = Liquid.calc_endmember_comp(mol_oxide_comp=liq_molox[liq2point], method='intrinsic', output_residual=True)
    Liq_molend[liq2point,:] = liq_moles_end[:]
    
#print(Liq_molend)

In [22]:
##############################################################################################################################
#                            Compute activities and chemical potentials of endmember components
#
# The following code creates empty arrays in which the variables for all the 
# melt chemical potentials (Liq_chempotentials), all melt centric information (Liq_melts_calcs_all), 
# the chemical potentials of the melt (muSiO2liq).
##############################################################################################################################

liq3points               = Liq_molend.shape[0]
temp_pts                 = t.shape[0]
pres_pts                 = p.shape[0]
Liq_chempotentials       = np.empty([liq3points,15])               
Liq_melts_calcs_all      = np.empty([liq3points,(1+1+1+16+15+15+15)])
muSiO2liq                = np.empty([liq3points,1])

for liq3point in range(liq3points):
    Liqchemactivity                         = Liquid.activity      (t[liq3point], p[liq3point], mol=Liq_molend[liq3point,:])  #Calculate the liquid activites using MELTS
    Liqchempotential                        = Liquid.chem_potential(t[liq3point], p[liq3point], mol=Liq_molend[liq3point,:])  #Calculate the liquid chemical potentials using MELTS
    Liq_chempotentials[liq3point,:]         = Liqchempotential[:]                #Write the liquid chemical potentials to the empty array we created above.
    muSiO2liq[liq3point,0]                  = Liq_chempotentials[liq3point,0] 
    Liq_melts_calcs_all[liq3point,0]        = Case[liq3point]            #Start filling in the array set up for all values; Here I start with the a reference to the case number
    Liq_melts_calcs_all[liq3point,1]        = tempc[liq3point]           #Fill in the temperature from the original input in Celcius.
    Liq_melts_calcs_all[liq3point,2]        = p[liq3point]               #Fill in the pressure in bars.
    Liq_melts_calcs_all[liq3point,3:19]     = liq_molox[liq3point,:]     #16 columns for mol fractions of 16 oxides
    Liq_melts_calcs_all[liq3point,19:34]    = Liq_molend[liq3point,:]    #15 columns for the fraction of each of the end members'FeCr2O4' 'FeAl2O4' 'Fe3O4' 'MgAl2O4' 'Fe2TiO4'
    Liq_melts_calcs_all[liq3point,34:49]    = Liqchempotential[:]        #15 columns for chemical potentials for each end member 'FeCr2O4' 'FeAl2O4' 'Fe3O4' 'MgAl2O4' 'Fe2TiO4'
    Liq_melts_calcs_all[liq3point,49:64]    = Liqchemactivity[:]         #15 columns for chemical activities for each end member 'FeCr2O4' 'FeAl2O4' 'Fe3O4' 'MgAl2O4' 'Fe2TiO4'

#np.savetxt("Liq_melts_calcs_Application_NaturalSamplesv2.csv", Liq_melts_calcs_all, delimiter=",")   
#print(muSiO2liq)

In [23]:
##############################################################################################################################
#                                          Calculated $\mu_{SiO_2}^0$ for experimental dataset
##############################################################################################################################

#Create empty array as a function of the dataset, with 15 columns
#that represent each of the liquid mol fraction endmembers
mu0_SiO2_molend      = np.empty([temp_pts,(15)])

#Fill the array with zeros
mu0_SiO2_molend.fill(0)
#Check that your array has all zeros
#print(mu0_SiO2_molend)

#Fill first column of array with 1, which means pure SiO2 to use in calculations of muSiO20
mu0_SiO2_molend[:, 0] = 1

#Create empty array as a function of the dataset, with 15 columns
mu0_SiO2all          = np.empty([temp_pts,15])

#Create empty array as a function of the dataset to store your muSiO2 values, only
mu0_SiO2             = np.empty([temp_pts,1])
#print(mu0_SiO2)          

#Calculate mu0_SiO2 for your entire dataset
muSipoints          = mu0_SiO2_molend.shape[0]       #Creating a unique looping variable (just in case I overwrite something from above)
#print(muSipoints)
for muSipoint in range(muSipoints):
    muSiO2_chemicalpotential    = Liquid.chem_potential(t[muSipoint], p[muSipoint], mol=mu0_SiO2_molend[muSipoint,:])  #Calculate the muSiO2 chemical potentials using MELTS
    mu0_SiO2all[muSipoint,:]    = muSiO2_chemicalpotential[:] #Store all values in an array, then (next line) write the first column into a new array.
    mu0_SiO2[muSipoint,:]       = mu0_SiO2all[muSipoint,0]#Store only the first column from the previous array

#print(mu0_SiO2)


In [24]:
##############################################################################################################################
#                          Calculate Silica Activity relative to Quartz (i.e. Quartz Affinity)
##############################################################################################################################
Quartz = modelDB.get_phase('Qz')

#The following lines of code set ups a looping variable and an empty array for the 
#chemical potential of quartz
qzpoints = tempc.shape[0]
mu0qz = np.empty([temp_pts,1])

for qzpoint in range(qzpoints):
    mu0qz[qzpoint,0] = Quartz.chem_potential(t[qzpoint], p[qzpoint])
      

#The following lines of code set up a looping variable and empty arrays for the 
#activity of quartz (aSiO2qz), the affinity of Qtz (affinityQtz), and     
aSiO2qz               = np.empty([temp_pts,1])
affinityQtz           = np.empty([temp_pts,1])

aSipoints          = mu0_SiO2_molend.shape[0]       #Creating a unique looping variable (just in case I overwrite something from above)

for aSipoint in range(aSipoints):
    affinityQtz[aSipoint,0]         = -(muSiO2liq[aSipoint] - mu0qz[aSipoint])

affinityQtzv2   = affinityQtz.astype(float)
RT =8.3143*t

#The following lines of code call the tridymite thermodynamic properties, 
#set up a looping variable and an empty array, solve and store values for the 
#chemical potential and affinities of tridymite

Trid         = modelDB.get_phase('Trd')
tridpoints   = tempc.shape[0]    
mu0trid      = np.empty([temp_pts,1])
affinitytrid = np.empty([temp_pts,1])

for tridpoint in range(tridpoints):
    mu0trid[tridpoint,0] = Trid.chem_potential(t[tridpoint], p[tridpoint])

for tridpoint in range(tridpoints):
    affinitytrid[tridpoint,0]         = -(muSiO2liq[tridpoint] - mu0trid[tridpoint])

    

In [25]:
##############################################################################################################################
#                 Save relevant liquid properties: mols endmembers, uSiO2, affinities of trid, quartz
##############################################################################################################################
Liq_melts_calcs_allv2 = np.empty([liq3points,(1+1+1+15+1+1+1)])

for liq3point in range(liq3points):
    Liq_melts_calcs_allv2[liq3point,0]    = Case[liq3point]
    Liq_melts_calcs_allv2[liq3point,1]    = tempc[liq3point]
    Liq_melts_calcs_allv2[liq3point,2]    = presMpa[liq3point]
    Liq_melts_calcs_allv2[liq3point,3:18] = Liq_molend[liq3point,:]
    Liq_melts_calcs_allv2[liq3point,18]   = muSiO2liq[liq3point,:]
    Liq_melts_calcs_allv2[liq3point,19]   = affinitytrid[liq3point,:]
    Liq_melts_calcs_allv2[liq3point,20]   = affinityQtzv2[liq3point,:]  
    
#Liquid end members from the MELTS algorithm
#'SiO2' 'TiO2' 'Al2O3' 'Fe2O3' 'MgCr2O4' 'Fe2SiO4' 'MnSi0.5O2' 'Mg2SiO4'
# 'NiSi0.5O2' 'CoSi0.5O2' 'CaSiO3' 'Na2SiO3' 'KAlSiO4' 'Ca3(PO4)2' 'H2O'
Liq_melts_calcs_all_df = pd.DataFrame({
    'Case':                               Liq_melts_calcs_allv2[:, 0].astype(int),
    'T °C':                               Liq_melts_calcs_allv2[:, 1],
    'P MPa':                              Liq_melts_calcs_allv2[:, 2],
    'Liq EndMember moles SiO2':           Liq_melts_calcs_allv2[:, 3],
    'Liq EndMember moles TiO2':           Liq_melts_calcs_allv2[:, 4],
    'Liq EndMember moles Al2O3':          Liq_melts_calcs_allv2[:, 5],
    'Liq EndMember moles Fe2O3':          Liq_melts_calcs_allv2[:, 6],
    'Liq EndMember moles MgCr2O4':        Liq_melts_calcs_allv2[:, 7],
    'Liq EndMember moles Fe2SiO4':        Liq_melts_calcs_allv2[:, 8],
    'Liq EndMember moles MnSi0.5O2':      Liq_melts_calcs_allv2[:, 9],
    'Liq EndMember moles Mg2SiO4':        Liq_melts_calcs_allv2[:,10],
    'Liq EndMember moles NiSi0.5O2':      Liq_melts_calcs_allv2[:,11],
    'Liq EndMember moles CoSi0.5O2':      Liq_melts_calcs_allv2[:,12],
    'Liq EndMember moles CaSiO3':         Liq_melts_calcs_allv2[:,13],
    'Liq EndMember moles Na2SiO3':        Liq_melts_calcs_allv2[:,14],
    'Liq EndMember moles KAlSiO4':        Liq_melts_calcs_allv2[:,15],
    'Liq EndMember moles Ca3(PO4)2':      Liq_melts_calcs_allv2[:,16],
    'Liq EndMember moles H2O':            Liq_melts_calcs_allv2[:,17],
    'mu SiO2 kJ':                         Liq_melts_calcs_allv2[:,18]/1000.0,
    'Affinity Tridymite kJ':              Liq_melts_calcs_allv2[:,19], 
    'Affinity Quartz kJ':                 Liq_melts_calcs_allv2[:,20]
})
Liq_melts_calcs_all_df.to_csv("Liq_calcs_OSaS_Benchmark_Input.csv", index=False)
Liq_melts_calcs_all_df.to_excel("Liq_calcs_OSaS_Benchmark_Input.xlsx", index=False)


In [26]:
##############################################################################################################################
#                                    MELTS OSaS (solve for $\mu_{O_2}$ and log$f_{O_2}$ )
#
# The following code solves for the chemical potential of O2 and then the logfO2. 
# The logfO2 is determined by subtracting the standard state chemical potentials of 
# oxygen from the chemical potential of oxygen (immediately below)
# 
##############################################################################################################################
muO2 = (2.*muFe3O4_correction[:,0] - 3.*muFe2SiO4[:,0] + 3.*muSiO2liq[:,0]) 

#Solve for standard state chemical potential of oxygen (mu0_O2)
t  =  t.astype(float)
tr = 298.15
hs = 23.10248*(t-tr) + 2.0*804.8876*(np.sqrt(t)-np.sqrt(tr)) \
    - 1762835.0*(1.0/t-1.0/tr) - 18172.91960*np.log(t/tr) \
    + 0.5*0.002676*(t*t-tr*tr)

ss = 205.15 + 23.10248*np.log(t/tr)  \
    - 2.0*804.8876*(1.0/np.sqrt(t)-1.0/np.sqrt(tr)) \
    - 0.5*1762835.0*(1.0/(t*t)-1.0/(tr*tr)) \
    + 18172.91960*(1.0/t-1.0/tr) + 0.002676*(t-tr)

mu0_O2= hs - t*ss

fO2Calc     = np.exp((muO2[:] - mu0_O2[:])/((8.314)*t[:]))
fO2Calc_2d  = np.reshape(fO2Calc, (temp_pts, 1))
logfO2Calc  = np.log10(fO2Calc)
logfO2Calc_MELTS = np.reshape(logfO2Calc, (temp_pts, 1))
print(logfO2Calc_MELTS)


###Save all details for MELTS OSaS
MELTS_OSaS = np.empty([temp_pts, 8])
for i in range(temp_pts):
    MELTS_OSaS[i,0]        = Case [i]
    MELTS_OSaS[i,1]        = logfO2Calc_MELTS[i]
    MELTS_OSaS[i,2]        = muO2[i]
    MELTS_OSaS[i,3]        = mu0_O2[i]
    MELTS_OSaS[i,4]        = muSiO2liq[i]
    MELTS_OSaS[i,5]        = muFe3O4_correction[i]
    MELTS_OSaS[i,6]        = muFe3O4[i]
    MELTS_OSaS[i,7]        = muFe2SiO4[i]

# FMQ buffer calculation
def QFM(T, P=1.0):
    """Frost 1991 equation, RiMG 25"""
    return -25096.3/T + 8.735 + 0.11*(P-1.0)/T
    
MELTS_OSaS_df = pd.DataFrame({
    'Case':                       MELTS_OSaS[:, 0].astype(int),
    'Sample':                     Abbrev,
    'logfO2 MELTS OSaS':          MELTS_OSaS[:, 1],
    'dFMQ':                       MELTS_OSaS[:, 1] - QFM(tempc + 273.15, P=350.0),
    'mu O2 kJ':                   MELTS_OSaS[:, 2]/1000.0,
    'mu0 O2 kJ':                  MELTS_OSaS[:, 3]/1000.0,
    'mu SiO2 liq kJ':             MELTS_OSaS[:, 4]/1000.0,
    'mu Fe3O4 corrected kJ':      MELTS_OSaS[:, 5]/1000.0,
    'mu Fe3O4 uncorrected kJ':    MELTS_OSaS[:, 6]/1000.0,
    'muFe2SiO4 kJ':               MELTS_OSaS[:, 7]/1000.0
})

MELTS_OSaS_df.to_excel("MELTSOSaS_OSaS_MAY_FO2_CALC.xlsx", index=False)
    



[[ -8.75102348]
 [ -9.46665191]
 [ -9.78940589]
 [ -9.26112764]
 [ -9.81182007]
 [ -9.37230184]
 [ -8.80163844]
 [ -8.74491622]
 [ -9.03238101]
 [-11.35583051]
 [-12.19334709]
 [-12.6065918 ]]
